In [ ]:
!pip install opencv-python-headless numpy matplotlib folium scipy opencv-python

In [ ]:
import cv2
import numpy as np
import math
import folium
from scipy.ndimage import gaussian_filter1d

cap = cv2.VideoCapture("video1.mp4")
ret, prev_frame = cap.read()
prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

prev_points = cv2.goodFeaturesToTrack(
    prev_gray,
    maxCorners=300,
    qualityLevel=0.3,
    minDistance=7,
    blockSize=7
)

lat, lon = 48.2658, 25.9184
trajectory = [(lat, lon)]

x_m, y_m = 0.0, 0.0
theta = 0.0

height = 25
gsd = 0.023

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    points_next, status, error = cv2.calcOpticalFlowPyrLK(
        prev_gray,
        gray,
        prev_points,
        None,
        winSize=(21, 21),
        maxLevel=2,
        criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 30, 0.01)
    )

    if points_next is None:
        prev_gray = gray
        prev_points = cv2.goodFeaturesToTrack(gray, maxCorners=300, qualityLevel=0.3, minDistance=7, blockSize=7)
        continue

    good_prev = prev_points[status == 1]
    good_next = points_next[status == 1]

    if len(good_prev) < 15:
        prev_gray = gray
        prev_points = cv2.goodFeaturesToTrack(gray, maxCorners=300, qualityLevel=0.3, minDistance=7, blockSize=7)
        continue

    M, inliers = cv2.estimateAffinePartial2D(good_prev, good_next, method=cv2.RANSAC, ransacReprojThreshold=3.0)
    if M is None:
        prev_gray = gray
        prev_points = good_next.reshape(-1, 1, 2)
        continue

    dx_pix = M[0, 2]
    dy_pix = M[1, 2]

    dtheta = math.atan2(M[1, 0], M[0, 0])
    theta += dtheta

    dx_m_local = dx_pix * gsd
    dy_m_local = dy_pix * gsd

    dx_m_global = dx_m_local * math.cos(theta) - dy_m_local * math.sin(theta)
    dy_m_global = dx_m_local * math.sin(theta) + dy_m_local * math.cos(theta)

    x_m += dx_m_global
    y_m += dy_m_global

    movement_north = -dy_m_global
    movement_east  =  dx_m_global

    delta_lat = movement_north / 111111.0
    delta_lon = movement_east  / (111111.0 * math.cos(math.radians(lat)))

    lat += delta_lat
    lon += delta_lon

    trajectory.append((lat, lon))

    if len(good_next) < 50:
        new_points = cv2.goodFeaturesToTrack(gray, maxCorners=300, qualityLevel=0.3, minDistance=7, blockSize=7)
        if new_points is not None:
            prev_points = new_points
        else:
            prev_points = good_next.reshape(-1, 1, 2)
    else:
        prev_points = good_next.reshape(-1, 1, 2)
    
    prev_gray = gray

cap.release()

trajectory_array = np.array(trajectory)
smoothed_lat = gaussian_filter1d(trajectory_array[:, 0], sigma=2)
smoothed_lon = gaussian_filter1d(trajectory_array[:, 1], sigma=2)
trajectory_smooth = list(zip(smoothed_lat, smoothed_lon))

m = folium.Map(location=[trajectory[0][0], trajectory[0][1]], zoom_start=18)

folium.PolyLine(
    trajectory, 
    color="lightblue", 
    weight=2.5, 
    opacity=1
).add_to(m)

folium.PolyLine(
    trajectory_smooth, 
    color="blue", 
    weight=2.5, 
    opacity=1,
    popup="Згладжена"
).add_to(m)

folium.Marker(trajectory[0], popup="Start", icon=folium.Icon(color='green')).add_to(m)
folium.Marker(trajectory[-1], popup="Finish", icon=folium.Icon(color='red')).add_to(m)
folium.Marker([48.2659, 25.9192], popup="Finish real", icon=folium.Icon(color='orange')).add_to(m)

m.save("drone_path.html")